In [99]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
sys.version
import datetime
import dateutil.parser
import json
import joblib
import numpy as np

#学習用データ，検証用データの読み込み先とそれぞれのdataframeの保存先に注意
forLearning = 0#学習用データなら1，検証用は0
if forLearning == 1:
    with open("/data/label-for-learning.json", "r") as f:
        df_all = pd.read_json(f)#学習用ラベル
elif forLearning == 0:
    with open("/data/label-for-evaluation.json", "r") as f:
        df_all = pd.read_json(f)#検証用評価データ
else:
    print('choose correct mode')
df={}
type_list=[]
type_dict={'type':'times'}
if forLearning == 1:
    timeRange = pd.date_range(start='2020-06-29 10:53:00',end='2020-07-06 04:28:00',freq='T')
elif forLearning == 0:
    timeRange = pd.date_range(start='2020-07-07 10:21:00',end='2020-07-13 11:20:00',freq='T')
#recipes毎にデータフレーム作成
recipes = df_all['recipes']
num=0
for i in range(len(df_all)):
    recipe=recipes[i]
    #時刻表記datetime型に
    raw_start_time = pd.to_datetime(recipe['started_at'])+datetime.timedelta(hours=9)
    start_time=raw_start_time.replace(second=0)#秒は無視して0に変更
    recipe['started_at']=(start_time)
    raw_stop_time = pd.to_datetime(recipe['stopped_at'])+datetime.timedelta(hours=9)
    stop_time=raw_stop_time.replace(second=0)
    recipe['stopped_at']=(stop_time)
    # type_listにtypeがなかったら
    if recipe['type'] not in type_list:
        # type_listに追加
        failureType = recipe['type']
        type_list.append(failureType)
        #print(type_list)
        type_dict[failureType]=0#あるfailureTypeが何回出てくるかカウント
        #print(type_dict)
        # 新しいtype用のdfの作成
        cols = list(recipe.keys())
        # 障害のtypeごとに列が違うため増やす必要がない
#         cols.extend(list(df_all.columns))
        colName = set(cols)
        df[failureType]=pd.DataFrame(index=timeRange,columns=colName)
        # time列作った方が取り回し良い
        df[failureType]['time_fromIndex'] = pd.to_datetime(df[failureType].index)

    # type 取得
    failureType = recipe['type']
    num = type_dict[failureType] + 1
    type_dict[failureType]=num
    #isFailureを代入
    df[failureType].loc[(df[failureType]['time_fromIndex']>=start_time) & (df[failureType]['time_fromIndex']<=stop_time), "isFailure"] = True
    # keyごとに代入
    for key in recipe.keys():
        # timeは情報量0なのでskip
        if key == "time":
            continue
        if type(recipe[key]) == dict :
            df[failureType].loc[(df[failureType]['time_fromIndex']>=start_time) & (df[failureType]['time_fromIndex']<=stop_time), key] = json.dumps(recipe[key])
        else:
            df[failureType].loc[(df[failureType]['time_fromIndex']>=start_time) & (df[failureType]['time_fromIndex']<=stop_time), key] = recipe[key]
#isFailureのNaNをFalseに置き換え
for key in df:
    df[key]['isFailure'] = df[key]['isFailure'].replace(np.nan,False)
    if forLearning == 1:#学習用データ
        df[key].to_pickle("/data/output-directory_all/df/label/{0}.pkl".format(key))
    elif forLearning == 0:#検証用データ
        df[key].to_pickle("/data/output-directory_random/df/label/{0}.pkl".format(key))
    else:
        print('something is wrong')

In [100]:
print(df_all['started_at'][0])
print(df_all['stopped_at'][0])

2020-07-07 10:20:20
2020-07-13 11:20:21


In [101]:
#to check
#print(type(recipes[0]['type']))
#type(recipes[0].keys())
#recipe.keys()
#type_dict
#df['tap-loss-start'].loc[df['tap-loss-start']['isFailure']==True]
for i in type_list:
    print(i)
    print('all',len(df[i]))
    print('isFailure True : ',len(df[i][df[i]['isFailure']==True]))

ixnetwork-traffic-start
all 8700
isFailure True :  0
node-down
all 8700
isFailure True :  78
node-up
all 8700
isFailure True :  75
tap-delay-start
all 8700
isFailure True :  229
tap-delay-stop
all 8700
isFailure True :  217
tap-loss-start
all 8700
isFailure True :  235
tap-loss-stop
all 8700
isFailure True :  234
interface-down
all 8700
isFailure True :  84
interface-up
all 8700
isFailure True :  81
ixnetwork-bgp-injection-start
all 8700
isFailure True :  171
ixnetwork-bgp-injection-stop
all 8700
isFailure True :  176
ixnetwork-bgp-hijacking-start
all 8700
isFailure True :  194
ixnetwork-bgp-hijacking-stop
all 8700
isFailure True :  194
ixnetwork-traffic-stop
all 8700
isFailure True :  2


In [97]:
path = "/data/output-directory_random/df/physical-infrastructure-bgpnw2/physical-infrastructure-bgpnw2.pkl"
path_vdevices = "/data/output-directory_random/df/virtual-infrastructure-bgpnw2/devices/IntGW-02_vi-devices.pkl"
df_check = pd.read_pickle(path_vdevices)
len(df_check)

9696

In [98]:
df_check

,id,name,flavor_id,flavor_name,flavor_ram,flavor_vcpus,flavor_disk,image_id,image_name,image_min_ram,...,metrics_disk-usage,metrics_disk-write-bytes,metrics_disk-write-bytes-rate,metrics_disk-write-requests,metrics_disk-write-requests-rate,metrics_memory,metrics_memory-resident,metrics_vcpus,project,time
0,fe46a471-965b-4447-9fa1-aa1721233627,IntGW-02,d7cfbc8a-0643-4587-b09a-bc47774c8c5b,bgp.router,8192.0,4.0,0.0,4cca1723-2565-4beb-9b89-0ba2f677b726,csr1kv-9.16.09.04,0.0,...,54960128.0,2.452098e+09,0.000000,392169.0,0.000000,8192.0,4684.0,4.0,bgpnw2,2020-06-29 10:53:00
1,fe46a471-965b-4447-9fa1-aa1721233627,IntGW-02,d7cfbc8a-0643-4587-b09a-bc47774c8c5b,bgp.router,8192.0,4.0,0.0,4cca1723-2565-4beb-9b89-0ba2f677b726,csr1kv-9.16.09.04,0.0,...,54960128.0,2.452122e+09,408.456268,392174.0,0.082807,8192.0,4684.0,4.0,bgpnw2,2020-06-29 10:54:00
2,fe46a471-965b-4447-9fa1-aa1721233627,IntGW-02,d7cfbc8a-0643-4587-b09a-bc47774c8c5b,bgp.router,8192.0,4.0,0.0,4cca1723-2565-4beb-9b89-0ba2f677b726,csr1kv-9.16.09.04,0.0,...,54960128.0,2.452122e+09,0.000000,392174.0,0.000000,8192.0,4684.0,4.0,bgpnw2,2020-06-29 10:55:00
3,fe46a471-965b-4447-9fa1-aa1721233627,IntGW-02,d7cfbc8a-0643-4587-b09a-bc47774c8c5b,bgp.router,8192.0,4.0,0.0,4cca1723-2565-4beb-9b89-0ba2f677b726,csr1kv-9.16.09.04,0.0,...,54960128.0,2.452122e+09,0.000000,392174.0,0.000000,8192.0,4684.0,4.0,bgpnw2,2020-06-29 10:56:00
4,fe46a471-965b-4447-9fa1-aa1721233627,IntGW-02,d7cfbc8a-0643-4587-b09a-bc47774c8c5b,bgp.router,8192.0,4.0,0.0,4cca1723-2565-4beb-9b89-0ba2f677b726,csr1kv-9.16.09.04,0.0,...,54960128.0,2.452147e+09,410.949497,392178.0,0.066220,8192.0,4684.0,4.0,bgpnw2,2020-06-29 10:57:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9691,fe46a471-965b-4447-9fa1-aa1721233627,IntGW-02,d7cfbc8a-0643-4587-b09a-bc47774c8c5b,bgp.router,8192.0,4.0,0.0,4cca1723-2565-4beb-9b89-0ba2f677b726,csr1kv-9.16.09.04,0.0,...,54960128.0,2.546764e+09,822.885485,408699.0,0.000000,8192.0,4684.0,4.0,bgpnw2,2020-07-06 04:24:00
9692,fe46a471-965b-4447-9fa1-aa1721233627,IntGW-02,d7cfbc8a-0643-4587-b09a-bc47774c8c5b,bgp.router,8192.0,4.0,0.0,4cca1723-2565-4beb-9b89-0ba2f677b726,csr1kv-9.16.09.04,0.0,...,54960128.0,2.546813e+09,822.885485,408708.0,0.150313,8192.0,4684.0,4.0,bgpnw2,2020-07-06 04:25:00
9693,fe46a471-965b-4447-9fa1-aa1721233627,IntGW-02,d7cfbc8a-0643-4587-b09a-bc47774c8c5b,bgp.router,8192.0,4.0,0.0,4cca1723-2565-4beb-9b89-0ba2f677b726,csr1kv-9.16.09.04,0.0,...,54960128.0,2.546818e+09,68.293656,408709.0,0.016706,8192.0,4684.0,4.0,bgpnw2,2020-07-06 04:26:00
9694,fe46a471-965b-4447-9fa1-aa1721233627,IntGW-02,d7cfbc8a-0643-4587-b09a-bc47774c8c5b,bgp.router,8192.0,4.0,0.0,4cca1723-2565-4beb-9b89-0ba2f677b726,csr1kv-9.16.09.04,0.0,...,54960128.0,2.546818e+09,0.000000,408709.0,0.000000,8192.0,4684.0,4.0,bgpnw2,2020-07-06 04:27:00
